In [ ]:
# variables with suffix '_n' denote that nudged data are being used
# variables with prefix 'd' denote derivative

# dependencies
using Base.Iterators
using MovingWeightedLeastSquares
using Plots
using Cubature
using Interpolations

# initialization of PyPlot plotting backend
pyplot();

In [ ]:
# initialize the dataset in this Jupyter notebook cell

# from, to, step
from = -1.95;
to = 1.95;
step = 0.975/2^3;
# a random number in interval (-ng/2, ng/2) will be added to each x in xs
ng = .2;
# function to be approximated
f = (x, y) -> exp(-(x^2 + y^2));
df = (x, y) -> 4 * x * y * exp(-(x^2 + y^2));
# weight function
w = (d, e) -> exp(-d^2/0.28^2);
# degree of used polynomial for approximation
deg = 4;
# eps of the weight function
eps = 3*step;
# attempts for time measurement
time_tries = 10;

In [ ]:
rg = from:step:to;
tozipxs = collect(from:step:to);
xs = transpose(hcat(collect.(collect(product(tozipxs,tozipxs)))[:]...));

# comment the line below, if repeated evaluation with the same nudges is needed
rnd_n = ng * rand(size(xs)) - ng/2;
xs_n = xs .+ rnd_n;

fs = [f(xs[i, :]...) for i in 1:size(xs, 1)];
fs_n = [f(xs_n[i, :]...) for i in 1:size(xs_n, 1)];
dfs = [df(xs[i, :]...) for i in 1:size(xs_n, 1)];
dfs_n = [df(xs_n[i, :]...) for i in 1:size(xs_n, 1)];
size(xs, 1)

In [ ]:
# creation of approximation objects
kd = mwls_kd(xs, fs, eps, w, maxDegree = deg);
kd_n = mwls_kd(xs_n, fs_n, eps, w, maxDegree = deg);
cll = mwls_cll(xs, fs, eps, w, maxDegree = deg);
cll_n = mwls_cll(xs_n, fs_n, eps, w, maxDegree = deg);

In [ ]:
function plot_func(xs::Vector{T}, ys::Vector{T}, zs::Vector{T}, savename::String) where {T <: Real}
    p = plot(xs, ys, zs, xlabel = "x", ylabel = "y", linetype = :surface)
    savefig(p, "$savename.pdf")
    p
end

In [ ]:
function plot_func(xs::Array{T}, ys::Vector{T}, savename::String) where {T <: Real}
    plot_func(xs[:, 1], xs[:, 2], ys, savename)
end

In [ ]:
function plot_func(b::Real, e::Real, f, savename::String; step::Real = 0.1)
    tozip = collect(b:step:e)
    xs = transpose(hcat(collect.(collect(product(tozip,tozip)))[:]...))
    plot_func(xs[:, 1], xs[:, 2], [f(xs[i, 1], xs[i, 2]) for i in 1:size(xs, 1)], savename);
end

In [ ]:
function plot_func(b::Real, e::Real, f::MwlsObject, ref, savename::String; step::Real = 0.1)
    tozip = collect(b:step:e)
    xs = transpose(hcat(collect.(collect(product(tozip,tozip)))[:]...))
    ys = [norm(f(xs[i, :]) - ref(xs[i, :]...)) for i in 1:size(xs, 1)]
    plot_func(xs[:, 1], xs[:, 2], ys, savename);
end

In [ ]:
function plot_func(b::Real, e::Real, f, ref, savename::String; step::Real = 0.1)
    tozip = collect(b:step:e)
    xs = transpose(hcat(collect.(collect(product(tozip,tozip)))[:]...))
    ys = [norm(f[xs[i, :]...] - ref(xs[i, :]...)) for i in 1:size(xs, 1)]
    plot_func(xs[:, 1], xs[:, 2], ys, savename);
end

In [ ]:
function plot_dfunc_err(b::Real, e::Real, f::MwlsObject, ref, savename::String; step::Real = 0.1)
    tozip = collect(b:step:e)
    xs = transpose(hcat(collect.(collect(product(tozip,tozip)))[:]...))
    ys = [norm(mwls_diff(f, xs[i, :], (1, 1)) - ref(xs[i, :]...)) for i in 1:size(xs, 1)]
    plot_func(xs[:, 1], xs[:, 2], ys, savename)
end

In [ ]:
# graph of the original function
plot_func(from, to, f, "orig")

In [ ]:
plot_func(from, to, df, "derived")

In [ ]:
function errfunc(obj::MwlsObject, reffunc::Function, b, e)
    hcubature(x -> abs(obj(x) - reffunc(x[1], x[2]))/(e - b)^2, [b, b], [e, e], abstol = 1e-1)
end

In [ ]:
function derrfunc(obj::MwlsObject, reffunc::Function, b, e)
    hcubature(x -> abs(mwls_diff(obj, x, (1, 1)) - reffunc(x[1], x[2]))/(e - b)^2, [b, b], [e, e], abstol = 1e-1)
end

In [ ]:
plot_func(from, to, cll, f, "cll-err")

In [ ]:
# error of the approximation
errfunc(cll, f, from, to)

In [ ]:
plot_dfunc_err(from, to, kd, df, "dkd-err")

In [ ]:
derrfunc(kd, df, from, to)

In [ ]:
plot_dfunc_err(from, to, kd_n, df, "dkd-err-n")

In [ ]:
# error of the approximation
errfunc(kd_n, f, from, to)

In [ ]:
plot_dfunc_err(from, to, cll_n, df, "dcll-err-n")

In [ ]:
derrfunc(cll_n, df, from, to)

In [ ]:
function interpolations_mass_approx(obj, xs::Array{T}) where {T <: Real}
    [obj[xs[i, :]...] for i in 1:size(xs, 1)]
end

In [ ]:
function interpolations_errfunc(obj, reffunc, b, e)
    hcubature(x -> abs(obj[x...] - reffunc(x[1], x[2]))/(e - b)^2, [b, b], [e, e], abstol = 1e-2)
end

In [ ]:
# init of the interpolation object from Interpolations.jl
itp = scale(interpolate([f(x1, x2) for x1 in rg, x2 in rg], BSpline(Cubic(Line())), OnGrid()), rg, rg);

In [ ]:
plot_func(from, to, itp, f, "bsplines-err")

In [ ]:
interpolations_errfunc(itp, f, from, to)